# 챗봇 대화

### skt/kogpt2-base-v2 모델 사용


In [1]:
# 필요한 라이브러리 설치
!pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:


import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
import random

# 샘플 대화 데이터
conversations = [
    {
        "user": "안녕하세요",
        "bot": "안녕하세요! 무엇을 도와드릴까요?"
    },
    {
        "user": "오늘 날씨가 어떤가요?",
        "bot": "죄송하지만 실시간 날씨 정보는 제공할 수 없어요. 날씨 앱을 확인해보시는 것을 추천드립니다."
    },
    {
        "user": "추천할 만한 영화가 있나요?",
        "bot": "어떤 장르를 선호하시나요? 액션, 로맨스, 코미디 등 취향을 알려주시면 더 좋은 추천을 드릴 수 있어요."
    },
    {
        "user": "액션 영화를 좋아해요",
        "bot": "액션 영화라면 '탑건: 매버릭', '미션 임파서블' 시리즈, '존 윅' 시리즈를 추천드려요. 모두 박진감 넘치는 액션으로 유명합니다."
    },
    {
        "user": "감사합니다",
        "bot": "천만에요! 다른 궁금한 것이 있으시면 언제든 물어보세요."
    },
    {
        "user": "프로그래밍 공부 방법을 알려주세요",
        "bot": "프로그래밍 공부는 꾸준한 연습이 중요해요. 기초 문법부터 시작해서 작은 프로젝트를 만들어보시고, 온라인 강의나 책을 활용하시면 좋습니다."
    }
]

# KoGPT-2 사용 (한국어 지원)
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 특수 토큰 추가
tokenizer.add_special_tokens({
    'pad_token': '<pad>',
    'bos_token': '<s>',
    'eos_token': '</s>',
    'sep_token': '<sep>'
})
model.resize_token_embeddings(len(tokenizer))

# 대화 데이터 전처리
def preprocess_conversations(conversations):
    processed_data = []

    for conv in conversations:
        # 대화 형식: <s>사용자: {user_input}<sep>봇: {bot_response}</s>
        text = f"<s>사용자: {conv['user']}<sep>봇: {conv['bot']}</s>"
        processed_data.append(text)

    return processed_data

# 데이터셋 클래스
class ChatDataset(Dataset):
    def __init__(self, conversations, tokenizer, max_length=128):
        self.data = preprocess_conversations(conversations)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()
        }

# 데이터 로더 생성
dataset = ChatDataset(conversations, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 훈련 설정
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 훈련 루프
model.train()
for epoch in range(10):
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Average Loss: {total_loss/len(dataloader):.4f}')

# 대화 생성 함수
def generate_response(user_input, max_length=100):
    model.eval()

    # 입력 형식 맞추기
    input_text = f"<s>사용자: {user_input}<sep>봇: "
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.8,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.9
        )

    # 응답 추출
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 봇 응답 부분만 추출
    if "봇: " in generated_text:
        response = generated_text.split("봇: ")[-1].strip()
        if "</s>" in response:
            response = response.split("</s>")[0].strip()
        return response
    else:
        return "죄송합니다. 응답을 생성할 수 없습니다."

# 대화형 테스트
def chat_interface():
    print("챗봇과 대화를 시작합니다. '종료'를 입력하면 대화가 끝납니다.")

    while True:
        user_input = input("\n사용자: ")
        if user_input.lower() in ['종료', 'quit', 'exit']:
            print("대화를 종료합니다.")
            break

        response = generate_response(user_input)
        print(f"봇: {response}")

# 테스트 실행
test_inputs = [
    "안녕하세요",
    "오늘 기분이 좋아요",
    "맛있는 음식 추천해주세요",
    "고마워요"
]

print("챗봇 테스트:")
for test_input in test_inputs:
    response = generate_response(test_input)
    print(f"사용자: {test_input}")
    print(f"봇: {response}")
    print("-" * 50)

# 대화형 인터페이스 실행 (주석 해제하여 사용)
chat_interface()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 1, Average Loss: 10.0765
Epoch 2, Average Loss: 1.8974
Epoch 3, Average Loss: 1.2286
Epoch 4, Average Loss: 0.8684
Epoch 5, Average Loss: 0.6985
Epoch 6, Average Loss: 0.5498
Epoch 7, Average Loss: 0.4479
Epoch 8, Average Loss: 0.3745
Epoch 9, Average Loss: 0.3196
Epoch 10, Average Loss: 0.2511
챗봇 테스트:
사용자: 안녕하세요
봇: 안녕하세요! 어떤 장르를 선호하시나요? 액션 영화를 좋아하시나요? 액션, 로맨스, 코미디 등 취향을 알려주시면 더 좋은 추천을 드릴 수 있어요.
--------------------------------------------------
사용자: 오늘 기분이 좋아요
봇: 안녕하세요!
--------------------------------------------------
사용자: 맛있는 음식 추천해주세요
봇: 감사합니다
--------------------------------------------------
사용자: 고마워요
봇: 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에요! 천만에
--------------------------------------------------
